<a href="https://colab.research.google.com/github/AbdulMoiz-MMY/Agentic_AI_Learning/blob/main/Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install --upgrade --quiet google-genai

In [12]:
from google.colab import userdata

GEMINI_API_KEY : str = userdata.get('GOOGLE_API_KEY')

if GEMINI_API_KEY:
  print("GEMINI_API_KEY FOUND")
else:
  print("GEMINI_API_KEY NOT FOUND")

GEMINI_API_KEY FOUND


In [13]:
from google import genai
from google.genai import Client

client: Client = genai.Client(
    api_key=GEMINI_API_KEY,
)

model: str = "gemini-2.0-flash-exp"

In [14]:
import time

def upload_video(video_file_name):
  video_file = client.files.upload(path=video_file_name)
  while video_file.state == "PROCESSING":
      print('Waiting for video to be processed.')
      time.sleep(10)
      video_file = client.files.get(name=video_file.name or "")

  if video_file.state == "FAILED":
    raise ValueError(video_file.state)
  print(f'Video processing complete: ' + (video_file.uri or ""))

  return video_file



In [15]:
magician_video = upload_video('magician.mp4')

Waiting for video to be processed.
Video processing complete: https://generativelanguage.googleapis.com/v1beta/files/bjsv5wakg8en


In [16]:
from google.genai.types import Content, Part
prompt = """For each scene in this video,
            generate captions that describe the scene along with any spoken text placed in quotation marks.
            Place each caption into an object with the timecode of the caption in the video.
         """

video = magician_video

response = client.models.generate_content(
    model=model,
    contents=[
        Content(
            role="user",
            parts=[
                Part.from_uri(
                    file_uri=video.uri or "",
                    mime_type=video.mime_type or ""),
                ]),
        prompt,
    ]
)



In [17]:
from IPython.display import Markdown
display(Markdown(response.text))


```json
[
  {
    "timecode": "00:00",
    "caption": "A black and white car drives on a grey track under a clear blue sky. The car has a number '9' on its side and a spoiler on the back. A hillside is visible in the background."
  },
  {
    "timecode": "00:01",
   "caption": "A black and white car continues driving on a grey track under a clear blue sky. The car has a number '9' on its side and a spoiler on the back. A hillside is visible in the background."
  },
  {
    "timecode": "00:02",
    "caption": "A black and white car keeps driving on a grey track under a clear blue sky. The car has a number '9' on its side and a spoiler on the back. A hillside is visible in the background."
  },
   {
    "timecode": "00:03",
   "caption": "A black and white car continues driving on a grey track under a clear blue sky. The car has a number '9' on its side and a spoiler on the back. A hillside is visible in the background."
  },
  {
    "timecode": "00:04",
   "caption": "A black and white car keeps driving on a grey track under a clear blue sky. The car has a number '9' on its side and a spoiler on the back. A hillside is visible in the background."
  }
]
```